# Chess Gameplay

First import our `chess_gameplay` module.

In [3]:
from pathlib import Path
import sys

sys.path.append(".")
sys.path.append(f"{Path.cwd()}/utils")
print(sys.path)

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/root/chess-hackathon-4/.venv/lib/python3.10/site-packages', '.', 'utils', '.', '/root/chess-hackathon-4/utils']


In [5]:
import chess_gameplay as chg

## Playing a game

Let's take a look at how games will be played in the tournament. Before running this cell, open the `demo.png` file alongside so you can watch the action!

In [10]:
# Instantiate agents and record team names. Note we're instantiating Agents with no arguments. These agents have been
# passed no models to inform their selections, so they will play purely random moves.

agents = {'white': chg.Agent(), 'black': chg.Agent()}
teams = {'white': 'Team White', 'black': 'Team Black'}

# Then call the `play_game` function. Note that we're playing here to a maximum depth of 5 moves each. In the
# tournament we will be playing to a maximum depth of 50 moves. We are also passing "poseval"=True which means 
# we will use StockFish to evaluate the board state after each move. These evaluations are used to update the 
# eval bar on the left side of the board rendering in `demo.png`. StockFish will be constrained by a time limit
# of 2 seconds and a depth limit of 25.

game_result = chg.play_game(
    agents, 
    teams, 
    max_moves=10, 
    min_seconds_per_move=0.2, 
    verbose=True, 
    poseval=True, 
    image_path="demo.png"
)

# Run this cell. A file `demo.png` will be saved to this repo directory which you can open and watch as it is 
# updated with moves from the game. The game may end in a checkmate, in which case the winner will recieve 1 point 
# and the loser will receieve 0 points. If the game ends in a draw or a stalemate, both will receieve 0 points. If 
# the maximum number of moves is reached without a conclusion to the game, the StockFish evaluations of the final 
# board state are used as the points for each Agent. For each pairing in the tournament, teams will play once as 
# white and once as black. The winner of the pairing will be the team with the highest score summed over the two 
# games. In the event of a draw, the pairing will be played again until a winner is declared.

white: e3
black: c5
white: d4


unhandled exception during asyncio.run() shutdown
task: <Task finished name='Task-596' coro=<wait_for() done, defined at /usr/lib/python3.10/asyncio/tasks.py:392> exception=EngineTerminatedError('engine process died unexpectedly (exit code: -9)')>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/tasks.py", line 432, in wait_for
    await waiter
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/tasks.py", line 435, in wait_for
    return fut.result()
  File "/root/chess-hackathon-4/.venv/lib/python3.10/site-packages/chess/engine.py", line 1242, in analyse
    await analysis.wait()
  File "/root/chess-hackathon-4/.venv/lib/python3.10/site-packages/chess/engine.py", line 2771, in wait
    return await self._finished
chess.engine.EngineTerminatedError: engine process died unexpectedly (exit code: -9)


KeyboardInterrupt: 

## Agents using models

Until you have trained a model and saved a checkpoint, you will not be able to run the following cell, but you can see how your model will be called and passed to an Agent to play with.

In [11]:
import torch
import yaml

# Your model must be imported exactly as follows; from a module called "model" (a file called "model.py") and with
# the class name "Model".

from model import Model

# All necessary arguments for your model to initialize with must be saved in a YAML file called "model_config.yaml"
# so that your model can be instantiated exactly as follows. Your model must NOT require any initialization arguments
# besides those described in your "model_config.yaml" file.

model_config = yaml.safe_load(open("model_config.yaml"))
model = Model(**model_config)

# Your model checkpoint must be called "checkpoint.pt" and must be a dictionary-like object with your model weights
# stored at the key "model" so that it can be loaded into your model exactly as follows.

device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = torch.load("checkpoint.pt", map_location=device)
model.load_state_dict(checkpoint["model"])

# Note: when you load your model weights you may see the following warning. You can safely ignore this warning.

ignore = """
/root/.chess/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
"""

In [12]:
# The model is passed as the first positional argument to the Agent, and is then available to the agent to use for
# selecting moves in the game.

agents = {'white': chg.Agent(model), 'black': chg.Agent(model)}
teams = {'white': 'Team White', 'black': 'Team Black'}

game_result = chg.play_game(
    agents, 
    teams, 
    max_moves=100, 
    min_seconds_per_move=0, 
    verbose=True, 
    poseval=True, 
    image_path="demo.png"
)

white: d3
black: e5
white: Na3
black: f5
white: h3
black: Qf6
white: d4
black: h5
white: Bg5
black: Kd8
Max moves reached.
White score: 0.532, Black score: 0.468
